In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from IPython.display import Markdown as md


In [ ]:
#hide
import numpy as np
import numpy as onp
import autograd 
path_assets = './assets/'

## Building a computation graph 

Computation graphs have many uses. Here I'll be presenting it as used in autodifferentiation systems.

For this post, I'll assume you have some familiarity with backpropagation and autodifferentiation (also known as autodiff). A autodiff package for Python is the [autograd](https://github.com/HIPS/autograd) package. [Autodidact](https://github.com/mattjj/autodidact) package is a somewhat simplified version of `autograd`. 

### What are we trying to find?


Say we have a function $f$ we want to differentiate. This function takes some number of arguments: one, two, three... however many. The aim is to find $df/dv$, where $v$ is one of these arguments to $f$.  

Here we define a function in two variables: x and y. $$f(x,y) = \log(x^2) + y^2 + xy$$

In [ ]:
def f(x,y):    return np.log(x**2) + y**2 + x*y

The `autograd` package creates a function `grad` that takes `f` as an input and the argument number (`argnum`) that you wish to differentiate with respect to. So to find $df/dx$, you'd put in `argnum=0`, and for $df/dy$ you'd put `argnum=1`, since we use zero-indexing and $x$ is the first variable and $y$ is the second variable. Then `grad` returns a function to find the gradient: here either $df/dx$ or $df/dy$. 

What's the structure of `grad`? Like most functions that return functions, it has a nested structure: it returns a function `gradfun` that in turn returns the gradient. 

To illustrate this further here's a super-basic implementation for `grad` specifically for our function `f` above. Instead of doing true autodifferentiation work here, we just return the analytic gradient $df/dx = 2x + \log(x^2) + y$ and $df/dy = 2y + x$. Autodiff packages obviously don't do this. 

In [ ]:
import numpy as np
def grad(f, argnum = 0): 
    """Returns a function that finds the gradient"""
    def gradfun(*args, **kwargs):
        """Returns the actual gradient """
        if len(args) != 0: x,y = args
        if len(kwargs) != 0: x,y = kwargs.values()
        # Dummy values. Returns correct gradient only for our function f above. 
        # Use these values until we calculate the true ones using autodiff. 
        #### remove this code once true code written
        if   argnum == 0: return 2*x * np.log(x**2) + y  # df/dx
        elif argnum == 1: return 2*y + x                 # df/dy
        #### 
        # true autograd code goes here 
        ####
    return gradfun
# example usage
dfdx = grad(f, argnum = 0)
dfdy = grad(f, argnum = 1)
print("dfdx", dfdx(1,2))      # call gradient w/out keywords, values go into *args    in gradfun 
print("dfdy", dfdy(x=13,y=4)) # call gradient with  keywords, values go into **kwargs in gradfun 

dfdx 2.0
dfdy 21


So that's our goal. We are trying to build this `grad` function properly, following the structure defined above. 

### Building a computation graph 

Vital to autodiff packages (and backpropagation in general) is the automatic construction of a computation graph. The computation graph breaks down functions into simplified expressions and keeps track of the order in which they occur. This lets you implement the backprogagation algorithm. This works (roughly) by starting at the end node (the node that returns a scalar),  having a variable to keep track of the gradient, and then by visiting each node (in  reverse topological order) and updating the global gradient, based on the vector-jacobian product for that node type. But we're getting ahead of ourselves here. 

The thing we are interested here in how this computation graph gets constructed. Let's break it down.

#### Basics

Say you had some expression, like $ (4 \times 5) + 2 - 4$. I'm sure you know the answer to this, but how would a computer work it out? 

Answer: order of operations. You may remember something about order of operations being important to make sure you get the right answer. I learnt the acronym [BODMAS](https://www.mathsisfun.com/operation-order-bodmas.html) in primary school

Python has its own order of operations too, governed by the hierarchy of [operator precedence](https://docs.python.org/3/reference/expressions.html#operator-precedence). Operator precendence determines the exact order that Python breaks down an expression. For example: assignment operators `:=` go first, `or` goes before `and` which goes before `not`, and multiplication and division go somewhere near the end.

Guess what? Write it out as nodes and links, and you get a computation graph. Each node is an action, like $\times 3, +5, \log$, and links to determine the order of the actions. The graph doesn't have any loops, which is nice. 

Here's the idea. Each node of the graph is a simple operation, like $+, x$, or $\log$. So working out its derivative is easy. For backpropagation, start at the top node: the one that holds the final answer. The gradient at each point on the computation graph is called the local gradient. Start at the head node and combine local gradients together until you reach a leaf node. The gradient at the leaf node is $df/dx$: the answer you seek. 

### Constructing the computation graph 

Now we need a way to build the computation graph. How do we do this? 

Let's do it `autograd` style. They do it in quite a clever way. Typically functions are made up of operators from the `numpy` package. What they do is create a copy of the `numpy` package (called `autograd.numpy`) that behaves exactly like `numpy`, except it keeps track of gradients and builds this computation graph as each function is called. They then write `import autograd.numpy as np` at the top of scripts, so that functions use the `autograd.numpy` version instead of `numpy`. 

Let's work with an example. Say we had the following function: 
$$ logistic(z) = \frac{1}{1 + \exp(-z)} $$

We would implement this in code like this

In [ ]:
def logistic(z): return 1 / (1 + np.exp(-z))


`numpy` uses operator overloading, meaning it replaces $+, \times, / $ with its own equivalents `np.add`, `np.multiply`, `np.divide` and so on. It does this defining the methods  `__add__`,`__mult__`,`__div__` in the `numpy.ndarray` class. If you pass in a `ndarray` to `logistic(z)`, it will know to use `np.add` when it encounters a `+` sign. The effect of this is that `logistic(z)` gets transformed into something like this: 

In [ ]:
def logistic2(z): return np.reciprocal(np.add(1, np.exp(np.negative(z))))

Let's see how Python breaks down this expression. Breaking it down has the same effect as constructing a number of intermediate variables, one after each operation, where each intermediate variable stores the result of a simple operation with the previous one. These simple operations are called **primitives** and they are important later. 

Let's call the intermediate variables $t_1, t_2, t_3...$, the input to the function $z$, and the final value $y$. 

In [ ]:
def logistic3(z): 
    t1 = np.negative(z) 
    t2 = np.exp(t1)
    t3 = np.add(1, t2)
    y = np.reciprocal(t3) 
    return y

We want to turn `logistic3` into a computation graph, with nodes and links between them. Here's what this graph looks like.

<img src="assets/node_tree.svg" width="240"/>

We will below define a class called `Node` and each node in the graph will be a member of this class. Each node links together by having a `parents` attribute that stores the nodes pointing to it in the graph. The leaf nodes above are $1$ and $z$, and the root node is $y$. The leaf nodes are typically constants or variables passed into the function, while the root node is the scalar-valued output of the function. Leaf nodes do not have parents. 

Below is a representation of the computational graph using Nodes. The numbers in `value` indicate the value of that intermediate variable. The function was given $z=1.5$ as an input and returns $y=0.818$. 

<img src="assets/node_tree.png" width="800"/>

Let's confirm we get the same answer. 

In [ ]:
np.round(logistic3(1.5),3)  # gives 0.818

0.818

#### Constructing the Node class, version 1

Now we can construct a first version of the Node class. For each Node, we need at least `value`, a function (`fun`) and `parents`. Let's create an tuple called `recipe` that we store `fun` and `value` in. 

We also will create a function called `initialise_root` that starts off the graph. A root of the tree doesn't have any parents, its function is just the identity function, and it has no value. 

In [ ]:
class Node:
    """A node in a computation graph."""
    def __init__(self, value, fun, parents):
        self.parents = parents
        self.recipe = (fun, value)
        # my additions
        self.value = value
        self.fun = fun 

    def initialize_root(self):
        self.parents = []
        self.recipe = (lambda x: x, None)

Now we have the Node class, we could manually build a computational graph if we wanted to. Let's create a `Node` for each intermediate variable. (We don't create a Node for $1$ or other scalars). 

In [ ]:
val_z = 1.5 
z = Node(val_z, None, [])
val_t1 = np.negative(val_z)
t1 = Node(val_t1,np.negative, [z])
val_t2 = np.exp(val_t1)
t2 = Node(val_t2, np.exp, [t1])
val_t3 = np.add(val_t2, 1)
t3 = Node(val_t3, np.add, [t2])
val_y = np.reciprocal(val_t3)
y = Node(val_y, np.reciprocal, [t3])
print(round(y.value,3)) # same answer as before

0.818


Creating the computational graph this way is both manual and clunky. In the next article we will learn how to build it automatically. 

### Creating a new version of Numpy

It would be great if numpy created a node for each intermediate variable and added it to our graph. But it won't do that. So it's time to make our own version of Numpy so we can trace the flow of execution and create our computation graph. 

First we'll delete the reference to our current numpy and import it under a new name, `onp`

try:     del np
except:  pass 
import numpy as _np, numpy as onp

Throughout this section, I'll refer to the original numpy as `onp`, and the version we are building as `anp`. 

There's a few things we have to do: 

* create new versions of `onp` functions. For example, we need to create a function `anp.add` that does everything `onp.add` does, but it also adds a node to the computation graph when called. Same with `anp.multiply`, `anp.divide` etc. 
* overload operators like $+, \times, /$ so that they use the `anp` versions: `anp.add`, `anp.multiply`, `anp.divide`, by defining functions like `__add__`, `__mul__`, `__div__`. 

Each numpy operation is wrapped in a *primitive*. 


### Resources

I used the following resources to put this document together. 

Lecture slides by Roger Grosse: https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()